In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib qt

In [2]:
import os
os.chdir("..")

In [3]:

import glob


In [4]:
print(os.getcwd())

c:\Users\eac84\Universiteit Antwerpen\Thermography - Biomedical thermography\src\DeipFlap


## Find H5 files

In [4]:
root = "I:/3_phd/Edgar/DIEP FLAP/*.H5"
H5_fns = glob.glob( root )
for i,fn in enumerate(H5_fns): print(i,fn)

0 I:/3_phd/Edgar/DIEP FLAP\20190921_1--PRE-UZA-20-09-19.H5
1 I:/3_phd/Edgar/DIEP FLAP\20190921_2-INTRA-UZA-20-09-19.H5
2 I:/3_phd/Edgar/DIEP FLAP\20190921_3-INTRA2-UZA-20-09-19-(enkel-venae-open).H5
3 I:/3_phd/Edgar/DIEP FLAP\20190921_4-INTRA-UZA-20-09-19-(alles-open).H5
4 I:/3_phd/Edgar/DIEP FLAP\20190921_5-INTRA-UZA-20-09-19-(alles-open-+beter-zicht).H5
5 I:/3_phd/Edgar/DIEP FLAP\20190921_6-INTRA-UZA-20-09-19-.H5
6 I:/3_phd/Edgar/DIEP FLAP\20190921_na-flap-inset.H5
7 I:/3_phd/Edgar/DIEP FLAP\20190921_per-op-dissectie-en-beide-open.H5
8 I:/3_phd/Edgar/DIEP FLAP\20191120_INTRALinks.H5
9 I:/3_phd/Edgar/DIEP FLAP\20191120_INTRARechts.H5
10 I:/3_phd/Edgar/DIEP FLAP\20191120_post.H5
11 I:/3_phd/Edgar/DIEP FLAP\20191120_PRE.H5
12 I:/3_phd/Edgar/DIEP FLAP\20200117_anastomose-li-flap-op-re-borst.H5
13 I:/3_phd/Edgar/DIEP FLAP\20200117_anastomose-re-flap-op-li-borst.H5
14 I:/3_phd/Edgar/DIEP FLAP\20200117_li-flap;-perfo-lager-in-flap-dan-op-eerste-tekening.H5
15 I:/3_phd/Edgar/DIEP FLAP\202001

In [33]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

In [62]:
filename = H5_fns[11]

with h5py.File(filename, 'r') as fh:
    
    print(fh.keys())
    im_data = fh["im_data"][:]

<KeysViewHDF5 ['im_data']>


In [63]:
print(im_data.shape)

plt.imshow(np.hstack(im_data[[0,20,-1]]), cmap="jet")

(27, 480, 640)


## Select Frames to Annotate 

In [5]:
#root = "C:\\Users\\eac84\\Desktop\\IRT Surgery"
root = "T:/ProjectData/DIEP FLAP/"
root = "I:/3_phd/Edgar/DIEP FLAP/*.H5"
xvi_fns = glob.glob( root + "*")
len(xvi_fns)

0

In [ ]:
import glob
import ThemoData

In [ ]:
 data = ThermoData(fn)

In [ ]:
plt.figure()
plt.title("Q: Quit \n S:Save Frame\n N:Next Movie\n")
plt.axis("off")

In [ ]:
xvi_fns = glob.glob( root + "\\*\\*.H5")
for fn in xvi_fns[5:22]:
    data = ThermoData(fn)
    select_dataset_frames(fn, data, out_dir="./training")

## Annotate Data

In [ ]:
#from thermography_python.io import *
from Annotation.image_annotator import *
from thermography_python.base import *
import glob
import matplotlib.pyplot as plt

In [ ]:
root = "./"
fns = glob.glob(root +  "*.H5")
fns

In [ ]:

an = Annotate(fns,exclude_labeled=False, run=False)
an.image_dataset = "im_data"
an.label_dataset = "labels"
an.open_window()


## Train Network

In [5]:
import torch
import glob as glob
import matplotlib.pyplot as plt

In [14]:
from retna.networks import *
from retna.training import *
from retna.data_loader import *
from retna.view import *


In [125]:
#model = torch.load(".\\models\\flap_model.pt", map_location=torch.device('cpu'))
#model = torch.load('./models/Flap+Pref3.pt',  map_location=torch.device('cpu'))
model2 = torch.load('./models/Flap4channel1.pt',  map_location=torch.device('cpu'))


In [7]:
in_channel = 1
out_channel = 2
h_chans = [8,8,8,8]

model = Retna_V1(in_channel, out_channel, h_chans)


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
from im_proc import *
from Segment import *
from registration import *
from read_data import *

In [61]:
fns = np.array(fns)[[11, 18,24,39,46]][[1]]

In [10]:
def draw_labels_1(im,classes):

    colors = np.array([[0,0,0],[0,0,1],[0,1,0],[1,0,0],[0,1,1]])
    colors = np.array(colors)*255

    imx = np.tile(im[...,None],[1,1,3])

    idx = classes[0]*0
    for n,cls in enumerate(classes):

        cls = bi_dilate(cls,1,1)
        if n==0: cls = cls & ~bi_erode(cls,5,0.5)
        if n==2: cls = bi_dilate(cls,2,1) & ~bi_erode(cls,1,0.5)
        idx[cls]=n+1

    L_rgb = colors[idx]

    draw = merge_layers(imx, L_rgb, a=0.5)

    return draw




def draw_labels_ontop(im, classes, tx=-1, ax=None, a=0.3):

    colors = [[0,0,0],[0,0,1],[0,1,0],[1,0,0],[0,1,1]]
    colors = np.array(colors)*255

    imx = im[(tx-3):(tx)].mean(axis=0)  
    imx = np.tile(imx[...,None],[1,1,3])
    
    #d_labels = labels/np.max(labels) * 255
    idx = classes[0]*0
    for n,cls in enumerate(classes):

        cls = bi_dilate(cls,1,1)
        if n==0: cls = cls & ~bi_erode(cls,5,0.5)
        if n==2: cls = bi_dilate(cls,2,1) & ~bi_erode(cls,1,0.5)

        idx[cls]=n+1

    L_rgb = colors[idx]

    draw = merge_layers(imx, L_rgb, a=0.5)
    L_rgb = draw_labels_annotations(classes)
    draw = merge_layers(imx, L_rgb, a=1)

    return draw


def draw_labels_annotations(classes):

    draw = classes[0]*0
    draw = np.tile(draw[...,None],[1,1,3])

    draw = draw_cross_navel(classes,draw)
    draw = draw_box_pref(classes, draw)

    return draw

def draw_cross_navel(classes,draw):

    bx = classes[1]
    if not np.any(bx): return draw
    x_c, y_c = np.argwhere(bx).sum(0)/bx.sum()
    x_c, y_c = int(x_c), int(y_c)

    draw[(x_c):(x_c+1),:] = [0,120,0]
    draw[:,(y_c):(y_c+1)] = [0,120,0]

    return draw

def draw_box_pref(classes, draw):

    bx = classes[2]
    bx = get_largest(bx)
    if not np.any(bx): return draw
    x, y = np.argwhere(bx).T
    x_min, x_max = int(x.min()), int(x.max())
    y_min, y_max = int(y.min()), int(y.max())

    draw[(x_min):(x_max),(y_min-1):(y_min+1)] = [255,0,0]
    draw[(x_min):(x_max),(y_max-1):(y_max+1)] = [255,0,0]

    draw[(x_min-1):(x_min+1),(y_min):(y_max)] = [255,0,0]
    draw[(x_max-1):(x_max+1),(y_min):(y_max)] = [255,0,0]

    return draw



def merge_layers(imx, L_rgb, a=0.3):

    bx = (L_rgb>0).any(axis=2)

    imx[bx] = (1-a)*imx[bx] + a*L_rgb[bx]

    draw = imx.astype(np.uint8)

    return draw




In [48]:
root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
root = "I:/3_phd/Edgar/DIEP FLAP/PRE/*"


fns = glob.glob(root + ".H5")

for n, fn in enumerate(fns[:20]):
  print(".")
  with h5py.File(fn,"r") as fh:

    t = int(len(fh["im_data"])/2)

    if len(fh["im_data"])< 5: continue
    
    im = fh["im_data"][t]
    classes = fh["mask"][:]
    draw = draw_labels_1(im,classes)

    plt.figure()
    plt.imshow(draw)
    plt.title(str(n) + "" + fn)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [176]:
im = imx[190][:400,:550]

classes = model.segment(im,scale=0.4)
classes[classes<0.3] = 0
classes = classes**2

draw = np.vstack((np.hstack(classes[:2]),np.hstack(classes[2:])))

plt.figure()
plt.imshow(im)

plt.figure()
plt.imshow(draw,cmap="jet")

In [174]:
plt.imsave("prediction.jpg",draw)

In [135]:
root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
root = "I:/3_phd/Edgar/DIEP FLAP/PRE/*"

plt.close("all")
fns = glob.glob(root + ".H5")

for n, fn in enumerate(fns[11:12]):
  print(".")
  with h5py.File(fn,"r") as fh:

    t = int(len(fh["im_data"])/2)
   
    im = fh["im_data"][t]
    classes = model.segment(im,scale=0.3)>0.95
    draw = draw_labels_1(im,classes)

    plt.figure()
    plt.imshow(draw)
    plt.title(str(n) + "" + fn)

.


In [110]:
with h5py.File(fn,"r") as fh:
    imx = fh["im_data"][::2]

(480, 640)

In [119]:


im = imx[200]
classes = model.segment(im,scale=0.3)>0.9
draw = draw_labels_1(im,classes)
plt.imshow(im)
plt.figure()
plt.imshow(np.vstack(classes))
plt.figure()
plt.imshow(draw)

In [116]:
plt.imshow(model.segment(im,scale=0.3)[0])

In [120]:
#root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
root = "I:/3_phd/Edgar/DIEP FLAP/PRE/"
fns = glob.glob(root + "*.H5")

for fn in fns[:20]:

    with h5py.File(fn,"r+") as fh:

      im = fh["im_data"][:]

      if len(im)<50: continue

      classes = label_preop(im[15:],tx=-100)
      if "mask" in fh.keys(): del fh["mask"]
      fh["mask"] = classes

      draw = draw_labels_1(im,classes)

      plt.figure()
      plt.imshow(draw)
      plt.title(str(n) + "" + fn)

KeyboardInterrupt: 

In [80]:
with h5py.File(fn,"r+") as fh:

      im = fh["im_data"][:]

In [93]:
classes = label_preop(im[15:],tx=-100)


In [94]:
plt.imshow(np.vstack(classes))

In [101]:
root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
fns = glob.glob(root + "*pre*.H5")
len(fns)

18

In [100]:
root = "//143.129.24.212/ProjectData/3_phd/Edgar/DIEP FLAP/"
fns = glob.glob(root + "/pre/*.H5")

fns = np.array(fns)[[11]]
DataSet = H5_Handler(fns, datasets=["im_data","mask"], which="all")
Loader = DataLoader(DataSet)
Loader.dataset.outsize = [100,100]
Loader.dataset.scale_range = [.1,.8]
Loader.dataset.rand_flip = [0.5,0]
Loader.dataset.noise = True
Loader.dataset.noise_range = [-0.2,0.2]
Loader.dataset.noise_low  = -0.3
Loader.dataset.noise_high = 0.3
Loader.dataset.rand_noise = 0.5

In [130]:
plt.close("")
print_mosaic(Loader,model, m=3, n=3)

[36]	0.32107812
[370]	0.27652544
[456]	0.52755797
[21]	0.7426154
[16]	0.6481153
[374]	0.46485627
[537]	0.89141953
[20]	0.3893359
[527]	0.38102877


In [254]:
plt.imshow(random_ramp((20,20)))

In [239]:
def random_ramp(shape):
    a,b = np.random.randint(low=-1, high = 2, size =2)
    x = np.linspace(a,-a, shape[0])
    y = np.linspace(b,-b, shape[1])

    X, Y = np.meshgrid(x, y)

    return X*Y

In [77]:
###########################
datasets=["im_data","mask"]
Loader.dataset.channels[1] = None
Loader.dataset.threshold = None
Loader.dataset.outsize = [200,200]
Loader.dataset.expand_label = True
Loader.dataset.scale_range = [.3,.4]
Loader.dataset.remap_label = [[1,0,0,0],
                              [1,1,0,0],
                              [1,0,1,0],
                              [1,0,0,1] ]

In [90]:
Loader.dataset.dataset_names = ["im_data","im_data"]

In [129]:
model.train_model(Loader, num_epochs = 50)

Start Loss: 0.4946383833885193
Epoch 51/60	....................Loss: 0.461534	Caching improved model	
Epoch 52/60	....................Loss: 0.458164	Caching improved model	
Epoch 53/60	....................Loss: 0.430610	Caching improved model	
Epoch 54/60	....................Loss: 0.426483	Caching improved model	
Epoch 55/60	....................Loss: 0.434185	
Epoch 56/60	....................Loss: 0.449166	
Epoch 57/60	....................Loss: 0.455856	lr: 0.0008	
Epoch 58/60	....................Loss: 0.462011	
Epoch 59/60	....................Loss: 0.480601	
Epoch 60/60	....................Loss: 0.465636	lr: 0.00064	
Best val loss: 0.426483


## Load the Data

In [35]:
label_fn = glob.glob("C:/Users/eac84/Universiteit Antwerpen/Thermography - Biomedical thermography/src/training/*_label.tiff")
dataSet = Image_Handler(label_fn)
Loader = DataLoader(dataSet)

Loader.dataset.remap_label = [[1,0,0,0],
                              [1,0,1,0],
                              ]

[]

In [ ]:
root = "C:/Users/eac84/Desktop/Test_Flap_data/"
fns = glob.glob(root +  "2*.H5")
print(fns)
DataSet = H5_Handler(fns, datasets=["im_data","labels"], which="all")
Loader = DataLoader(DataSet)


In [ ]:
if 0:
    Loader.dataset.device="cpu"
    model = model.to("cpu")
else:
    Loader.dataset.device="cuda:0"
    model = model.to("cuda:0")

In [ ]:
in_channel = 1
out_channel = 4
h_chans = [16,16,16,16,16]
model = Retna_V1(in_channel, out_channel, h_chans)

[85]	[45]	[55]	[14]	[36]	[41]	[57]	[28]	[0]	[96]	[2]	[20]	[40]	[16]	[53]	[43]	[21]	[68]	[58]	[71]	[93]	[79]	[0]	[3]	[13]	

In [ ]:
print_comparisons(model,Loader)

In [ ]:
draw_model_params(model)

In [ ]:
Loader.dataset.reset_weights()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(".\\models\\flap_model.pt", map_location=torch.device('cpu'))
model  = model.to(device)

In [ ]:
#path = "./models"
#if not os.path.isdir(path): os.mkdir(path)
torch.save(model, './models/Flap4channel.pt')

## Sandbox

In [12]:

## Not sure how to add this to annotator 
fns = glob.glob(".\\training\\*_input.tiff")
cv2.namedWindow("", cv2.WINDOW_NORMAL)
cv2.resizeWindow("", 1400, 600)   

bad_names = []
for fn in fns:
    fn2 = fn.replace("input","label")
    if os.path.exists(fn2):
        im = io.imread(fn)
        im2 = io.imread(fn2)       
        draw = np.hstack((im,im2*100))
        cv2.imshow("",draw)   
        
        key = cv2.waitKeyEx(10000)
        if key == ord('s'):                
            print(".")
        
        if key == ord('b'):                
            bad_names.append( fn2 )

cv2.destroyAllWindows()   

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:449: error: (-215:Assertion failed) !winname.empty() in function 'cv::namedWindow'
